#Отчет об Rush00


In [ ]:
import re
from enum import Enum
import urllib
import requests
from bs4 import BeautifulSoup
import timeit
from functools import reduce
from typing import List, Dict
from collections import Counter, OrderedDict
from datetime import date

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')

Mounted at /content/drive


Вспомогательные методы

In [20]:
def mean(seq):
	if len(seq) == 0:
		return float('nan')
	return sum(seq) / len(seq)


def median(seq):
	if len(seq) == 0:
		return float('nan')
	seq = sorted(seq)
	return seq[len(seq) // 2] if len(seq) % 2 == 1 else (seq[len(seq) // 2 - 1] + seq[len(seq) // 2]) / 2


def variance(seq):
	if len(seq) == 0:
		return float('nan')
	return sum([(x - mean(seq)) ** 2 for x in seq]) / len(seq)


Класс Links

In [65]:
class Links:
	"""
	Analyzing data from links.csv
	"""
	def __init__(self, path_to_the_file):
		"""
		Put here any fields that you think you will need.
		"""
		data = []
		# проверка на открытие файла
		with open(path_to_the_file, 'r') as f:
			str_data = (f.read().lower())
		data = str_data.split('\n')
		next_data = []
		for i in data[1:len(data) - 1]:
			data = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", i)
			# next_data.append(movieId, imdbId, tmdbId)
			next_data.append(data)
		# print(next_data)
		self.data_file = next_data
		self.list_movies_data = []
		for i in range(0, 10):
			url = f'https://www.imdb.com/title/tt{self.data_file[i][1]}/'
			headers = {'User-Agent': 'My User Agent 1.0',
			'From': f'https://www.imdb.com/title/tt{self.data_file[i][1]}/'
			}
			page = urllib.request.urlopen(urllib.request.Request(url=url, headers=headers)).read()
			soup = BeautifulSoup(page, 'lxml')
			tags = soup.find_all('li', role='presentation', class_='ipc-metadata-list__item')
			for tag in tags:
				if (tag.text.find('Director') != -1):
					Director = tag.text.replace('Director', '')
					break
			for tag in tags:
				if (tag.text.find('Budget') != -1):
					Budget = tag.text.replace('Budget', '')
					break
			for tag in tags:
				if (tag.text.find('Also known as') != -1):
					film = tag.text.replace('Also known as', '')
					break
			for tag in tags:
				if (tag.text.find('Gross worldwide') != -1):
					Gross = tag.text.replace('Gross worldwide', '')
					break
			for tag in tags:
				if (tag.text.find('Runtime') != -1):
					Runtime = tag.text.replace('Runtime', '')
					break
			field_data = [self.data_file[i][1], film, Director, Budget, Gross, Runtime]
			self.list_movies_data.append(field_data)

	def get_imdb(list_of_movies, list_of_fields):
		"""
		The method returns a list of lists [movieId, field1, field2, field3, ...] for the list of movies given as the argument (movieId).
		For example, [movieId, Director, Budget, Cumulative Worldwide Gross, Runtime].
		The values should be parsed from the IMDB webpages of the movies.
		Sort it by movieId descendingly.
		"""
		list_movies_data = []
		for i in list_of_movies:
			url = f'https://www.imdb.com/title/tt{i}/'
			headers = {'User-Agent': 'My User Agent 1.0',
			'From': f'https://www.imdb.com/title/tt{i}/'
			}
			page = urllib.request.urlopen(urllib.request.Request(url=url, headers=headers)).read()
			soup = BeautifulSoup(page, 'lxml')
			tags = soup.find_all('li', role='presentation', class_='ipc-metadata-list__item')
			field_data = [i]
			for field in list_of_fields:
				for tag in tags:
					if (tag.text.find(field) != -1):
						field_value = tag.text.replace(field, '')
						break
				field_data.append(field_value)
			list_movies_data.append(field_data)
		imdb_info = list(sorted(list_movies_data, key=lambda x: (-int(x[0]))))
		return imdb_info

		
	def top_directors(self, n):
		"""
		The method returns a dict with top-n directors where the keys are directors and 
		the values are numbers of movies created by them. Sort it by numbers descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			list_movies.append([self.list_movies_data[i][2], self.list_movies_data[i][0]])
		a = sorted(list_movies)
		directors = dict()
		for i in a:
			directors[i[0]] = i[1]
		return directors
		
	def most_expensive(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are their budgets. Sort it by budgets descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			list_movies.append([self.list_movies_data[i][1], self.list_movies_data[i][3]])
		a = sorted(list_movies)
		budgets = dict()
		for i in a:
			budgets[i[0]] = i[1]
		return budgets
		
	def most_profitable(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are the difference between cumulative worldwide gross and budget.
		Sort it by the difference descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			gross = ''.join(x for x in self.list_movies_data[i][4] if x.isdigit())
			budget = ''.join(x for x in self.list_movies_data[i][3] if x.isdigit())
			list_movies.append([self.list_movies_data[i][1], int(gross) - int(budget)])
		a = sorted(list_movies, key = lambda x: -x[1])
		profits = dict()
		for i in a:
			profits[i[0]] = i[1]
		return profits
		
	def longest(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are their runtime. If there are more than one version â€“ choose any.
		Sort it by runtime descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			Runtime_min = int(int(self.list_movies_data[i][5].split(' ')[0]) * 60) + int(self.list_movies_data[i][5].split(' ')[2])
			field_data = [self.list_movies_data[i][1], self.list_movies_data[i][5], Runtime_min]
			list_movies.append(field_data)
		a = sorted(list_movies, key = lambda x: -x[2])
		runtimes = dict()
		for i in a:
			runtimes[i[0]] = i[1]
		return runtimes
		
	def top_cost_per_minute(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are the budgets divided by their runtime. The budgets can be in different currencies â€“ do not pay attention to it. 
		The values should be rounded to 2 decimals. Sort it by the division descendingly.
		"""
		list_movies = []
		if (n > len(self.list_movies_data) and n <= 0):
			ValueError('n > list')
		for i in range(0, n):
			Runtime_min = int(int(self.list_movies_data[i][5].split(' ')[0]) * 60) + int(self.list_movies_data[i][5].split(' ')[2])
			budget = ''.join(x for x in self.list_movies_data[i][3] if x.isdigit())
			field_data = [self.list_movies_data[i][1], round(int(budget) / Runtime_min, 2)]
			list_movies.append(field_data)
		a = sorted(list_movies, key = lambda x: -x[1])
		costs = dict()
		for i in a:
			costs[i[0]] = i[1]
		return costs

Проверка методов класса Links

In [67]:
link = Links("/content/drive/MyDrive/links.csv")
print(Links.get_imdb(['0114709'], ['Director', 'Budget', 'Gross worldwide', 'Runtime']))
%timeit(Links.get_imdb(['0114709'], ['Director', 'Budget', 'Gross worldwide', 'Runtime']))


[['0114709', 'John Lasseter', '$30,000,000 (estimated)', '$394,436,586', '1 hour 21 minutes']]
1 loop, best of 5: 735 ms per loop


In [60]:
n = 10
print(link.top_directors(n))
%timeit(link.top_directors(n))

{'Charles Shyer': '0113041', 'Forest Whitaker': '0114885', 'Howard Deutch': '0113228', 'Joe Johnston': '0113497', 'John Lasseter': '0114709', 'Martin Campbell': '0113189', 'Michael Mann': '0113277', 'Peter Hewitt': '0112302', 'Peter Hyams': '0114576', 'Sydney Pollack': '0114319'}
The slowest run took 6.36 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.68 µs per loop


In [61]:
print(link.most_expensive(n))
%timeit(link.most_expensive(n))

{'Bir oh desem': '$16,000,000 (estimated)', 'Bố Của Cô Dâu 2': '$30,000,000 (estimated)', 'Cái Chết Bất Ngờ': '$35,000,000 (estimated)', 'Fuego contra fuego': '$60,000,000 (estimated)', 'Golden Eye': '$60,000,000 (estimated)', 'Jumanji: Trò Chơi Bí Ẩn': '$65,000,000 (estimated)', 'Toy Story in 3-D': '$30,000,000 (estimated)', 'Сабріна': '$58,000,000 (estimated)', 'Старі буркуни розбушувалися': '$25,000,000 (estimated)', 'Том і Гек': '$58,000,000 (estimated)'}
The slowest run took 4.80 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.97 µs per loop


In [62]:
print(link.most_profitable(n))
%timeit(link.most_profitable(n))

{'Toy Story in 3-D': 364436586, 'Golden Eye': 292194034, 'Jumanji: Trò Chơi Bí Ẩn': 197821940, 'Fuego contra fuego': 127436818, 'Bir oh desem': 65452156, 'Bố Của Cô Dâu 2': 46594107, 'Старі буркуни розбушувалися': 46518503, 'Cái Chết Bất Ngờ': 29350171, 'Сабріна': -4327920, 'Том і Гек': -34079952}
10000 loops, best of 5: 55.1 µs per loop


In [63]:
print(link.longest(n))
%timeit(link.longest(n))

{'Fuego contra fuego': '2 hours 50 minutes', 'Golden Eye': '2 hours 10 minutes', 'Сабріна': '2 hours 7 minutes', 'Bir oh desem': '2 hours 4 minutes', 'Cái Chết Bất Ngờ': '1 hour 51 minutes', 'Bố Của Cô Dâu 2': '1 hour 46 minutes', 'Jumanji: Trò Chơi Bí Ẩn': '1 hour 44 minutes', 'Старі буркуни розбушувалися': '1 hour 41 minutes', 'Том і Гек': '1 hour 37 minutes', 'Toy Story in 3-D': '1 hour 21 minutes'}
100000 loops, best of 5: 18.6 µs per loop


In [64]:
print(link.top_cost_per_minute(n))
%timeit(link.top_cost_per_minute(n))

{'Jumanji: Trò Chơi Bí Ẩn': 625000.0, 'Том і Гек': 597938.14, 'Golden Eye': 461538.46, 'Сабріна': 456692.91, 'Toy Story in 3-D': 370370.37, 'Fuego contra fuego': 352941.18, 'Cái Chết Bất Ngờ': 315315.32, 'Bố Của Cô Dâu 2': 283018.87, 'Старі буркуни розбушувалися': 247524.75, 'Bir oh desem': 129032.26}
10000 loops, best of 5: 56.6 µs per loop


Класс Tags

In [75]:
class Tags:
	def __init__(self, path_to_the_file):
		data = []
		# проверка на открытие файла
		with open(path_to_the_file) as file:
			self.__tags: List[str] = []
			next(file)
			for line in file:
				_, _, tag, _ = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", line.strip())
				while re.match("\"(.*)\"", tag):
					tag = re.sub("\"(.*)\"", r"\1", tag)
				self.__tags.append(tag)
		
	
	def most_words(self, n):
		"""
		The method returns top-n tags with most words inside. It is a dict 
 		where the keys are tags and the values are the number of words inside the tag.
		Drop the duplicates. Sort it by numbers descendingly.
        """
		return dict(sorted({x: len(re.findall(r"\w+", x)) for x in set(self.__tags)}.items(), key=lambda x: -x[1])[:n])

	def longest(self, n):
		"""
		The method returns top-n longest tags in terms of the number of characters.
		It is a list of the tags. Drop the duplicates. Sort it by numbers descendingly.
		"""
		return dict(sorted({x: len(x) for x in set(self.__tags)}.items(), key=lambda x: -x[1])[:n])

	def most_words_and_longest(self, n):
		"""
		The method returns the intersection between top-n tags with most words inside and 
		top-n longest tags in terms of the number of characters.
		Drop the duplicates. It is a list of the tags.
		"""
		return list(set(self.most_words(n)).intersection(self.longest(n)))
        
	def most_popular(self, n):
		"""
		The method returns the most popular tags. 
		It is a dict where the keys are tags and the values are the counts.
		Drop the duplicates. Sort it by counts descendingly.
		"""
		return dict(Counter(self.__tags).most_common(n))
        
	def tags_with(self, word):
		"""
		The method returns all unique tags that include the word given as the argument.
		Drop the duplicates. It is a list of the tags. Sort it by tag names alphabetically.
		"""
		return sorted((filter(lambda x: re.search(r'\b{}\b'.format(word), x), set(self.__tags))))

Проверка методов класса Tags

In [76]:
tags = Tags("/content/drive/MyDrive/tags.csv")
print(tags.most_words(10))
%timeit(tags.most_words(10))

{'Something for everyone in this one... saw it without and plan on seeing it with kids!': 16, 'the catholic church is the most corrupt organization in history': 10, 'villain nonexistent or not needed for good story': 8, 'It was melodramatic and kind of dumb': 7, 'stop using useless characters for filler': 6, '06 Oscar Nominated Best Movie - Animation': 6, 'r:disturbing violent content including rape': 6, 'Academy award (Best Supporting Actress)': 5, 'stop looking at me swan': 5, 'based on a TV show': 5}
100 loops, best of 5: 2.46 ms per loop


In [77]:
print(tags.longest(10))
%timeit(tags.most_words(10))

{'Something for everyone in this one... saw it without and plan on seeing it with kids!': 85, 'the catholic church is the most corrupt organization in history': 63, 'villain nonexistent or not needed for good story': 48, 'r:disturbing violent content including rape': 43, '06 Oscar Nominated Best Movie - Animation': 41, 'stop using useless characters for filler': 40, 'Academy award (Best Supporting Actress)': 39, 'Oscar (Best Effects - Visual Effects)': 37, 'It was melodramatic and kind of dumb': 36, 'r:sustained strong stylized violence': 36}
100 loops, best of 5: 2.41 ms per loop


In [78]:
print(tags.most_words_and_longest(10))
%timeit(tags.most_words(10))

['Academy award (Best Supporting Actress)', '06 Oscar Nominated Best Movie - Animation', 'villain nonexistent or not needed for good story', 'It was melodramatic and kind of dumb', 'r:disturbing violent content including rape', 'the catholic church is the most corrupt organization in history', 'Something for everyone in this one... saw it without and plan on seeing it with kids!', 'stop using useless characters for filler']
100 loops, best of 5: 2.44 ms per loop


In [79]:
print(tags.most_popular(10))
%timeit(tags.most_popular(10))

{'In Netflix queue': 131, 'atmospheric': 36, 'superhero': 24, 'thought-provoking': 24, 'funny': 23, 'Disney': 23, 'surreal': 23, 'religion': 22, 'sci-fi': 21, 'dark comedy': 21}
1000 loops, best of 5: 493 µs per loop


In [80]:
print(tags.tags_with('a'))
%timeit(tags.tags_with('a'))

['Strangers on a Train', 'a clever chef rat', 'a dingo ate my baby', 'based on a TV show', 'based on a book', 'based on a play', 'based on a true story', 'building a family', 'coulda been a contender', 'start of a beautiful friendship']
100 loops, best of 5: 2.41 ms per loop


Класс Movie

In [49]:
import re
from functools import reduce
from typing import List, Dict
from collections import Counter, OrderedDict
from datetime import date


class Movie:
	def __init__(self, id, title, genres):
		self.id = id
		self.title = title
		try:
			self.year = int(re.sub('.*\(([0-9]{4})\).*', r'\1', title))
		except:
			self.year = 2000
		self.genres = [] if genres == '(no genres listed)' else genres.split('|')

class Movies:
	"""
	Analyzing data from movies.csv
	"""
	def __init__(self, path_to_the_file: str):
		"""
        Put here any fields that you think you will need.
        """
		with open(path_to_the_file) as file:
			self.__movies: Dict[Movie] = {}
			next(file)
			for line in file:
				movieId, title, genres = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", line.strip())
				while re.match("\"(.*)\"", title):
					title = re.sub("\"(.*)\"", r"\1", title)
				self.__movies[movieId] = (Movie(movieId, title, genres))

	def dist_by_release(self):
		"""
		The method returns a dict or an OrderedDict where the keys are years and the values are counts.
		You need to extract years from the titles. Sort it by counts descendingly.
		"""
		return dict(Counter(map(lambda x: x.year, self.__movies.values())).most_common())

	def dist_by_genres(self):
		"""
		The method returns a dict where the keys are genres and the values are counts.
		Sort it by counts descendingly.
		"""
		return dict(Counter(reduce(lambda a, b: a + b, map(lambda x: x.genres, self.__movies.values()))).most_common())

	def most_genres(self, n):
		"""
		The method returns a dict with top-n movies where the keys are movie titles and
		the values are the number of genres of the movie. Sort it by numbers descendingly.
		"""
		return dict(Counter({x.title: len(x.genres) for x in self.__movies.values()}).most_common(n))

	def __getitem__(self, item):
		return self.__movies[item]

Проверка методов класса Movie

In [50]:
movies = Movies("/content/drive/MyDrive/movies.csv")
print(movies.most_genres(10))
%timeit(movies.most_genres(10))

{'Rubber (2010)': 10, 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)': 8, 'Mulan (1998)': 7, 'Who Framed Roger Rabbit? (1988)': 7, 'Osmosis Jones (2001)': 7, 'Interstate 60 (2002)': 7, 'Robots (2005)': 7, 'Pulse (2006)': 7, 'Aqua Teen Hunger Force Colon Movie Film for Theaters (2007)': 7, 'Enchanted (2007)': 7}
100 loops, best of 5: 4.36 ms per loop


In [51]:
print(movies.dist_by_genres())
%timeit(movies.dist_by_genres())

{'Drama': 4361, 'Comedy': 3756, 'Thriller': 1894, 'Action': 1828, 'Romance': 1596, 'Adventure': 1263, 'Crime': 1199, 'Sci-Fi': 980, 'Horror': 978, 'Fantasy': 779, 'Children': 664, 'Animation': 611, 'Mystery': 573, 'Documentary': 440, 'War': 382, 'Musical': 334, 'Western': 167, 'IMAX': 158, 'Film-Noir': 87}
1 loop, best of 5: 555 ms per loop


In [52]:
print(movies.dist_by_release())
%timeit(movies.dist_by_release())

{2002: 311, 2000: 296, 2006: 295, 2001: 294, 2007: 284, 2009: 282, 2003: 279, 2004: 279, 2014: 278, 1996: 276, 2015: 274, 2005: 273, 2008: 269, 1999: 263, 1997: 260, 1995: 259, 1998: 258, 2011: 254, 2010: 247, 2013: 239, 1994: 237, 2012: 233, 2016: 218, 1993: 198, 1992: 167, 1988: 165, 1987: 153, 1990: 147, 1991: 147, 2017: 147, 1989: 142, 1986: 139, 1985: 126, 1984: 101, 1981: 92, 1980: 89, 1982: 87, 1983: 83, 1979: 69, 1977: 63, 1973: 59, 1978: 59, 1965: 47, 1971: 47, 1974: 45, 1976: 44, 1964: 43, 1967: 42, 1968: 42, 1975: 42, 1966: 42, 2018: 41, 1962: 40, 1972: 39, 1963: 39, 1959: 37, 1960: 37, 1955: 36, 1969: 35, 1961: 34, 1970: 33, 1957: 33, 1958: 31, 1953: 30, 1956: 30, 1940: 25, 1949: 25, 1954: 23, 1942: 23, 1939: 23, 1946: 23, 1951: 22, 1950: 21, 1947: 20, 1948: 20, 1941: 18, 1936: 18, 1945: 17, 1937: 16, 1952: 16, 1944: 16, 1938: 15, 1931: 14, 1935: 13, 1933: 12, 1934: 11, 1943: 10, 1932: 9, 1927: 7, 1930: 5, 1926: 5, 1924: 5, 1929: 4, 1928: 4, 1925: 4, 1923: 4, 1916: 4, 1920:

Класс Ratings

In [33]:
class Rating:
	def __init__(self, userId, movieId, rating, timestamp):
		self.userId = userId
		self.movieId = movieId
		self.userId = userId
		self.rating = float(rating)
		self.year = date.fromtimestamp(int(timestamp))


class Ratings:
	"""
    Analyzing data from ratings.csv
    """

	def __init__(self, path_to_the_ratings, path_to_movies):
		"""
        Put here any fields that you think you will need.
        """
		self.__ratings = Ratings.Movies(path_to_the_ratings, Movies(path_to_movies))
		self.__users = Ratings.Users(path_to_the_ratings, Movies(path_to_movies))

	@property
	def ratings(self):
		return self.__ratings

	@property
	def users(self):
		return self.__users

	def dist_by_year(self):
		"""
        The method returns a dict where the keys are years and the values are counts.
        Sort it by years ascendingly. You need to extract years from timestamps.
        """
		return self.__ratings.dist_by_year()

	def dist_by_rating(self):
		"""
        The method returns a dict where the keys are ratings and the values are counts.
     Sort it by ratings ascendingly.
        """
		return self.__ratings.dist_by_rating()

	def top_by_num_of_ratings(self, n):
		"""
        The method returns top-n movies by the number of ratings.
        It is a dict where the keys are movie titles and the values are numbers.
        Sort it by numbers descendingly.
        """
		return self.__ratings.top_by_num_of_ratings(n)

	def top_by_ratings(self, n, metric='average'):
		"""
        The method returns top-n movies by the average or median of the ratings.
        It is a dict where the keys are movie titles and the values are metric values.
        Sort it by metric descendingly.
        The values should be rounded to 2 decimals.
        """
		return self.__ratings.top_by_ratings(n, metric)

	#
	def top_controversial(self, n):
		"""
        The method returns top-n movies by the variance of the ratings.
        It is a dict where the keys are movie titles and the values are the variances.
      Sort it by variance descendingly.
        The values should be rounded to 2 decimals.
        """
		return self.__ratings.top_controversial(n)

	class Movies:
		def __init__(self, path_to_the_file, movies: Movies):
			self._movies = movies
			with open(path_to_the_file) as file:
				self._ratings: List[Rating] = []
				next(file)
				for line in file:
					userId, movieId, rating, timestamp = re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)", line.strip())
					self._ratings.append(Rating(userId, movieId, rating, timestamp))

		def dist_by_year(self):
			"""
            The method returns a dict where the keys are years and the values are counts.
            Sort it by years ascendingly. You need to extract years from timestamps.
            """
			return dict(Counter(map(lambda x: x.year, self._ratings)).most_common()[::-1])

		def dist_by_rating(self):
			"""
            The method returns a dict where the keys are ratings and the values are counts.
         Sort it by ratings ascendingly.
            """
			return dict(Counter(map(lambda x: x.rating, self._ratings)).most_common()[::-1])

		def top_by_num_of_ratings(self, n):
			"""
            The method returns top-n movies by the number of ratings.
            It is a dict where the keys are movie titles and the values are numbers.
            Sort it by numbers descendingly.
            """
			return {self._movies[x[0]].title: x[1] for x in
			        Counter(map(lambda x: x.movieId, self._ratings)).most_common(n)}

		def top_by_ratings(self, n, metric="average"):
			"""
            The method returns top-n movies by the average or median of the ratings.
            It is a dict where the keys are movie titles and the values are metric values.
            Sort it by metric descendingly.
            The values should be rounded to 2 decimals.
            """
			output = {}
			for elem in self._ratings:
				if elem.movieId not in output:
					output[elem.movieId] = []
				output[elem.movieId].append(elem.rating)
			if metric == "average":
				for movieId in output:
					output[movieId] = round(mean(output[movieId]), 2)
			elif metric == "median":
				for movieId in output:
					output[movieId] = round(median(output[movieId]), 2)
			else:
				return []
			return {self._movies[elem[0]].title: elem[1] for elem in sorted(output.items(), key=lambda x: -x[1])[:n]}

		def top_controversial(self, n):
			"""
            The method returns top-n movies by the variance of the ratings.
            It is a dict where the keys are movie titles and the values are the variances.
            Sort it by variance descendingly.
            The values should be rounded to 2 decimals.
            """
			output = {}
			for elem in self._ratings:
				if elem.movieId not in output:
					output[elem.movieId] = []
				output[elem.movieId].append(elem.rating)
			for movieId in output:
				output[movieId] = round(variance(output[movieId]), 2)
			return {self._movies[elem[0]].title: elem[1] for elem in sorted(output.items(), key=lambda x: -x[1])[:n]}

	class Users(Movies):
		"""
        In this class, three methods should work.
        The 1st returns the distribution of users by the number of ratings made by them.
        The 2nd returns the distribution of users by average or median ratings made by them.
        The 3rd returns top-n users with the biggest variance of their ratings.
        Inherit from the class Movies. Several methods are similar to the methods from it.
        """

		def dist_by_num_of_ratings(self):
			return dict(Counter(map(lambda x: x.userId, self._ratings)).most_common())

		def dist_by_ratings(self, metric="average"):
			output = {}
			for elem in self._ratings:
				if elem.userId not in output:
					output[elem.userId] = []
				output[elem.userId].append(elem.rating)
			if metric == "average":
				for userId in output:
					output[userId] = round(mean(output[userId]), 2)
			elif metric == "median":
				for userId in output:
					output[userId] = round(median(output[userId]), 2)
			else:
				return []
			return {self._movies[elem[0]].title: elem[1] for elem in sorted(output.items(), key=lambda x: -x[1])}

		def top_controversial(self, n):
			output = {}
			for elem in self._ratings:
				if elem.userId not in output:
					output[elem.userId] = []
				output[elem.userId].append(elem.rating)
			for userId in output:
				output[userId] = round(variance(output[userId]), 2)
			return dict(sorted(output.items(), key=lambda x: -x[1])[:n])

Проверка методов класса Ratings



In [53]:
ratings = Ratings("/content/drive/MyDrive/ratings.csv", "/content/drive/MyDrive/movies.csv")
print(ratings.users.top_controversial(5))
%timeit(ratings.users.top_controversial(5))

{'3': 4.26, '461': 3.1, '55': 3.09, '259': 2.94, '329': 2.92}
1 loop, best of 5: 400 ms per loop


In [54]:
print(ratings.users.dist_by_rating())
%timeit(ratings.users.dist_by_rating())

{0.5: 1370, 1.5: 1791, 1.0: 2811, 2.5: 5550, 2.0: 7551, 4.5: 8551, 3.5: 13136, 5.0: 13211, 3.0: 20047, 4.0: 26818}
10 loops, best of 5: 26.2 ms per loop


In [55]:
print(ratings.users.dist_by_num_of_ratings())
%timeit(ratings.users.dist_by_num_of_ratings())

{'414': 2698, '599': 2478, '474': 2108, '448': 1864, '274': 1346, '610': 1302, '68': 1260, '380': 1218, '606': 1115, '288': 1055, '249': 1046, '387': 1027, '182': 977, '307': 975, '603': 943, '298': 939, '177': 904, '318': 879, '232': 862, '480': 836, '608': 831, '600': 763, '483': 728, '590': 728, '105': 722, '19': 703, '305': 677, '489': 648, '111': 646, '438': 635, '217': 613, '140': 608, '477': 600, '555': 578, '91': 575, '28': 570, '219': 528, '534': 520, '89': 518, '64': 517, '226': 507, '561': 505, '18': 502, '525': 500, '57': 476, '381': 474, '368': 469, '509': 467, '469': 465, '560': 458, '462': 455, '292': 446, '21': 443, '597': 443, '42': 440, '160': 437, '294': 437, '580': 436, '596': 411, '202': 403, '275': 403, '517': 400, '45': 399, '156': 398, '514': 397, '391': 386, '567': 385, '357': 383, '103': 377, '339': 371, '62': 366, '199': 363, '125': 360, '51': 359, '132': 347, '66': 345, '313': 340, '200': 334, '221': 331, '6': 314, '453': 311, '50': 310, '425': 306, '428': 3

In [56]:
print(ratings.top_by_ratings(5))
%timeit(ratings.top_by_ratings(5))

{'The Jinx: The Life and Deaths of Robert Durst (2015)': 5.0, 'Galaxy of Terror (Quest) (1981)': 5.0, 'Alien Contamination (1980)': 5.0, "I'm the One That I Want (2000)": 5.0, 'Lesson Faust (1994)': 5.0}
10 loops, best of 5: 65.4 ms per loop


In [57]:
print(ratings.dist_by_rating())
%timeit(ratings.dist_by_rating())

{0.5: 1370, 1.5: 1791, 1.0: 2811, 2.5: 5550, 2.0: 7551, 4.5: 8551, 3.5: 13136, 5.0: 13211, 3.0: 20047, 4.0: 26818}
10 loops, best of 5: 25.6 ms per loop


In [58]:
print(ratings.top_controversial(5))
%timeit(ratings.top_controversial(5))

{"Ivan's Childhood (a.k.a. My Name is Ivan) (Ivanovo detstvo) (1962)": 5.06, 'Fanny and Alexander (Fanny och Alexander) (1982)': 5.06, 'Troll 2 (1990)': 4.5, 'Lassie (1994)': 4.0, 'Zed & Two Noughts, A (1985)': 4.0}
10 loops, best of 5: 156 ms per loop


In [59]:
print(ratings.top_by_num_of_ratings(5))
%timeit(ratings.top_by_num_of_ratings(5))

{'Forrest Gump (1994)': 329, 'Shawshank Redemption, The (1994)': 317, 'Pulp Fiction (1994)': 307, 'Silence of the Lambs, The (1991)': 279, 'Matrix, The (1999)': 278}
10 loops, best of 5: 28.5 ms per loop
